In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import time


dir_path = '../1. Original Data'
save_path = '../2. Prepared Data'
train_pattern = '/TRAIN_BD_'
test_pattern = '/TEST_BD_'
file_name = ['CONSUMO.csv','DEM_TIME.csv']

## <u> Bases de Usuários</u>:

In [4]:
#BASES USUARIO - PREPROCESSAMENTO
df_train_usr = pd.read_csv(dir_path+train_pattern+file_name[1]);
df_test_usr = pd.read_csv(dir_path+test_pattern+file_name[1]);
print(df_train_usr.info())
print(df_test_usr.info())

print('BASE TREINAMENTO -------------------------------------------------------------------')
print(df_train_usr.head())
print(df_train_usr.tail())
rate_missing_chave = 1 - len(df_train_usr.query('KEY == KEY'))/len(df_train_usr)
rate_missing_sexo = 1 - len(df_train_usr.query('SEXO == SEXO'))/len(df_train_usr)
rate_missing_nasc = 1 - len(df_train_usr.query('DTA_NASC == DTA_NASC'))/len(df_train_usr)
rate_missing_estado = 1 - len(df_train_usr.query('ESTADO == ESTADO'))/len(df_train_usr)
rate_missing_time = 1 - len(df_train_usr.query('TIME == TIME'))/len(df_train_usr)
print(rate_missing_sexo,rate_missing_nasc)

print('BASE TESTE -------------------------------------------------------------------------')
print(df_test_usr.head())
print(df_test_usr.tail())
rate_missing_sexo = 1 - len(df_test_usr.query('SEXO == SEXO'))/len(df_test_usr)
rate_missing_nasc = 1 - len(df_test_usr.query('DTA_NASC == DTA_NASC'))/len(df_test_usr)
print(rate_missing_sexo,rate_missing_nasc)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 951464 entries, 0 to 951463
Data columns (total 5 columns):
KEY         951464 non-null int64
SEXO        910294 non-null object
DTA_NASC    914441 non-null float64
ESTADO      951464 non-null object
TIME        951464 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 36.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407773 entries, 0 to 407772
Data columns (total 5 columns):
KEY         407773 non-null int64
SEXO        390241 non-null object
DTA_NASC    391901 non-null float64
ESTADO      407773 non-null object
TIME        407773 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 15.6+ MB
None
BASE TREINAMENTO -------------------------------------------------------------------
       KEY SEXO  DTA_NASC             ESTADO      TIME
0  2421280    M    1994.0       Minas Gerais  Cruzeiro
1  1604566    M    1990.0              Bahia  Flamengo
2   133472    M    1985.0     Rio de Janeiro

In [8]:
#TRATAMENTO DOS VALORES NULOS, PREENCHENDO COM A MÉDIA EM COLUNA OU PROPAGANDO VALOR ANTERIOR

df_train_usr.DTA_NASC.fillna(np.mean(df_train_usr.DTA_NASC),inplace=True)
df_train_usr.SEXO.fillna(method='ffill',inplace=True)

assert len(df_train_usr.query('DTA_NASC != DTA_NASC')) == 0
assert len(df_train_usr.query('SEXO != SEXO')) == 0

df_test_usr.DTA_NASC.fillna(np.mean(df_test_usr.DTA_NASC),inplace=True)
df_test_usr.SEXO.fillna(method='ffill',inplace=True)

assert len(df_test_usr.query('DTA_NASC != DTA_NASC')) == 0
assert len(df_test_usr.query('SEXO != SEXO')) == 0

#VERIFICACAO DE HIPOTESES

assert len(np.unique(df_train_usr.KEY)) == len(df_train_usr)
assert len(np.unique(df_test_usr.KEY)) == len(df_test_usr)
assert list(np.unique(df_train_usr.ESTADO)) == list(np.unique(df_test_usr.ESTADO))
assert list(np.unique(df_train_usr.TIME)) == list(np.unique(df_test_usr.TIME))
assert list(np.unique(df_train_usr.SEXO)) == list(np.unique(df_test_usr.SEXO))

In [9]:
#CRIACAO E ATUALIZACAO DE FEATURES
ano = time.localtime().tm_year
df_train_usr['IDADE'] = ano - df_train_usr.DTA_NASC
df_test_usr['IDADE'] = ano - df_test_usr.DTA_NASC

print(df_train_usr.head(2))
print(df_test_usr.head(2))

df_train_usr.drop('DTA_NASC',axis=1,inplace=True)
df_test_usr.drop('DTA_NASC',axis=1,inplace=True)

#PADRONIZACAO DOS TIMES

def standardize_team(team):
    if team.lower() == 'goiais': return 'goias'
    return team.lower().replace(' ','-')

df_train_usr.TIME = df_train_usr.TIME.apply(standardize_team)
df_test_usr.TIME = df_train_usr.TIME.apply(standardize_team)

print(df_train_usr.head(2))
print(df_test_usr.head(2))

       KEY SEXO  DTA_NASC        ESTADO      TIME  IDADE
0  2421280    M    1994.0  Minas Gerais  Cruzeiro   24.0
1  1604566    M    1990.0         Bahia  Flamengo   28.0
       KEY SEXO  DTA_NASC               ESTADO TIME  IDADE
0  2837889    M    1988.0  Rio Grande do Norte  ABC   30.0
1  2066186    M    1985.0  Rio Grande do Norte  ABC   33.0
       KEY SEXO        ESTADO      TIME  IDADE
0  2421280    M  Minas Gerais  cruzeiro   24.0
1  1604566    M         Bahia  flamengo   28.0
       KEY SEXO               ESTADO      TIME  IDADE
0  2837889    M  Rio Grande do Norte  cruzeiro   30.0
1  2066186    M  Rio Grande do Norte  flamengo   33.0


In [10]:
print(df_train_usr.info())
print(df_test_usr.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 951464 entries, 0 to 951463
Data columns (total 5 columns):
KEY       951464 non-null int64
SEXO      951464 non-null object
ESTADO    951464 non-null object
TIME      951464 non-null object
IDADE     951464 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 36.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407773 entries, 0 to 407772
Data columns (total 5 columns):
KEY       407773 non-null int64
SEXO      407773 non-null object
ESTADO    407773 non-null object
TIME      407773 non-null object
IDADE     407773 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 15.6+ MB
None


In [11]:
df_train_usr.to_csv(save_path+'/20180924_user_train.csv')
df_test_usr.to_csv(save_path+'/20180924_user_test.csv')